In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
# Download the FTSE 100 data from Yahoo Finance
df = yf.download("^FTSE", start="2022-01-01", end="2024-07-07")
df.to_csv("./Data/FTSE100.csv")

In [ ]:
df = yf.download("RR.L", start="2022-01-01", end="2024-07-07")
df.to_csv("./Data/RollsRoyce.csv")

# Get TCFD Score from Report of Companies, using ChatGPT to extract the score

In [ ]:
import pdfplumber
import glob

# Find all PDF files in the Data folder recursively
pdf_files = glob.glob('./Data/*.pdf', recursive=True)

# Open the PDF file
for pdf_file in pdf_files:
    with pdfplumber.open(pdf_file) as pdf:
        # Extract text from each page
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
        # Print the extracted text (for verification)
        print(text)
        # Save the extracted text to a text file
        with open(pdf_file.replace('.pdf', '.txt'), encoding='utf-8', mode='w') as f:
            f.write(text)


In [ ]:
# Load stock data
import pandas as pd
df = pd.read_csv("./Data/RollsRoyce.csv", index_col='Date', parse_dates=True)

In [ ]:
import matplotlib.pyplot as plt
# Plot the stock price
plt.figure(figsize=(10, 6))
plt.plot(df['Close'], label='Close')
# Identify trend changes, plot red dots for peaks and green dots for troughs
trend = new_trend = None
# Fixed window size for the moving average
window_size = 20
change_dates = []
# if the price is above the moving average by 20% or more, it is an uptrend
for i in range(window_size, len(df) - window_size):
    if df['Close'][i] > df['Close'][i - window_size:i].mean() * 1.1:
        new_trend = 'up'
    elif df['Close'][i] < df['Close'][i - window_size:i].mean() * 0.9:
        new_trend = 'down'
    if new_trend != trend:
        change_dates.append(df.index[i])
        plt.plot(df.index[i], df['Close'][i], 'ro' if new_trend == 'up' else 'go')
        trend = new_trend
plt.title('Rolls Royce stock price')
plt.legend()
plt.show()
change_dates



In [ ]:
# Perform a linear regression to find the relationship between the FTSE 100 and Rolls Royce stock
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
# Load FTSE 100 data
df_ftse = pd.read_csv("./Data/FTSE100.csv", index_col='Date', parse_dates=True)
df = pd.read_csv("./Data/RollsRoyce.csv", index_col='Date', parse_dates=True)

assert df.index[0] == df_ftse.index[0], 'The start date of the two datasets should be the same'
if len(df) > len(df_ftse):
    df = df.loc[:df_ftse.index[-1]]
elif len(df) < len(df_ftse):
    df_ftse = df_ftse.loc[:df.index[-1]]

stock_return = df['Close'].pct_change().dropna()
ftse_return = df_ftse['Close'].pct_change().dropna()
X = ftse_return
Y = stock_return

# Perform join on the two dataframes
print("Y fisrt and last date: ", Y.index[0], Y.index[-1])
print("X fisrt and last date: ", X.index[0], X.index[-1])
X = ftse_return.tolist()
Y = stock_return.tolist()
assert len(X) == len(Y), 'The length of X and Y should be the same '+ str(len(X))+' '+str(len(Y))

X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()
print(model.summary())  

# Step 4: Calculate Abnormal Returns
stock_returns_predicted = model.predict(X)
abnormal_returns = stock_return[:len(stock_returns_predicted)] - stock_returns_predicted

# Plot the abnormal returns
plt.figure(figsize=(10, 6))
plt.plot(abnormal_returns, label='Abnormal Returns')
plt.title('Rolls Royce Abnormal Returns')
plt.legend()
plt.show()
